In [7]:
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm  

image_folder = 'redfin_images'

model = YOLO("best.pt")

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_name in tqdm(image_files, desc="Processing Images"):
    image_path = os.path.join(image_folder, image_name)
    
    image = cv2.imread(image_path)
    
    results = model.predict(image)

    if len(results[0].boxes) == 0:
        os.remove(image_path)
print("Processing complete.")
